In [1]:
import os

os.environ['create_image']='True'
os.environ['repository']='docker.io/romeokienzler'
os.environ['version']='0.1'
os.environ['name']='claimed-sim-wrf'

In [2]:
if bool(os.environ.get('create_image',False)):
    docker_file="""
    FROM romeokienzler/mywrfcontainerimage
    SHELL ["/bin/tcsh", "-c"]
    WORKDIR /wrf/WRF
    RUN ./clean -a
    RUN ./configure
    RUN ./compile em_real >&! foo
    WORKDIR /wrf/WPS
    RUN ./configure
    RUN ./compile >&! foo
    """
    with open("Dockerfile", "w") as text_file:
        text_file.write(docker_file)

    !docker build -t `echo $name`:`echo $version` .
    #!docker tag `echo $name`:`echo $version` `echo $repository`/`echo $name`:`echo $version`
    #!docker push `echo $repository`/`echo $name`:`echo $version`
    !rm Dockerfile
else: 
    None

Sending build context to Docker daemon   25.6kB
Step 1/8 : FROM romeokienzler/mywrfcontainerimage
 ---> 7cf57e8f908b
Step 2/8 : WORKDIR /wrf/WRF
 ---> Using cache
 ---> 96d7a5ddcc69
Step 3/8 : RUN ./clean -a
 ---> Using cache
 ---> bb1a1805bf3a
Step 4/8 : RUN ./configure
 ---> Running in dcd9136a53bd
checking for perl5... no
checking for perl... found /usr/bin/perl (perl)
 
*****************************************************************************
No environment variable NETCDF set.
Stopping
*****************************************************************************
 
The command '/bin/sh -c ./configure' returned a non-zero code: 5


In [ ]:
#! /bin/sh
ls
docker   run   -it   --name   teachme   7cf57e8f908b   /bin/tcsh
ls -ls
cd WRF
./clean -a
./configure
./compile em_real >&! foo
ls -ls main/*.exe
cd ../WPS
./configure
./compile >&! foo
ls -ls *.exe
cp namelist.wps namelist.wps.original
cp /wrf/wrfinput/namelist.wps.docker namelist.wps
./geogrid.exe
ls -ls geo_em.d01.nc
./link_grib.csh /wrf/wrfinput/fnl
cp ungrib/Variable_Tables/Vtable.GFS Vtable
./ungrib.exe
ls -ls FILE*
./metgrid.exe
ls -ls met_em.*
cd ../WRF/test/em_real
ln -sf ../../../WPS/met_em* .
cp namelist.input namelist.input.original
cp /wrf/wrfinput/namelist.input.docker namelist.input
mpirun -np 2 ./real.exe
tail rsl.out.0000
ls -ls wrfinput_d01 wrfbdy_d01
mpirun -np 3 ./wrf.exe &
tail rsl.out.0000
ls -ls wrfo*